<h2>Stock Market Transformer Model</h2>


In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import TorchNormalizer
import torch.nn as nn
import torch

<h3>Data</h3>

In [44]:
import yfinance as yf
data = yf.download(tickers="AAPL", period='max', interval='1d', groupby='ticker', auto_adjust='True')
data.head()

#create random data


[*********************100%***********************]  1 of 1 completed


AttributeError: module 'numpy' has no attribute 'rand'

In [58]:
random_timeseries = pd.DataFrame(
    dict(
    date=[x for x in range(1,1001)],
    open=np.random.rand(1000) + 0.2,
    close=np.random.rand(1000) + 1000,
        high=np.random.rand(1000) + 18,
        low = np.random.rand(1000) + 2,
        volume=np.random.rand(1000) + 2
    ))
random_timeseries["target"] = random_timeseries.close.shift(-1)
random_timeseries.fillna(3)
random_timeseries

,date,open,close,high,low,volume,target
0,1,0.292381,1000.811746,18.377700,2.486988,2.152189,1000.456786
1,2,1.075209,1000.456786,18.152134,2.630768,2.505689,1000.974828
2,3,1.107432,1000.974828,18.528306,2.929421,2.145361,1000.318113
3,4,0.410290,1000.318113,18.563099,2.818635,2.960683,1000.661179
4,5,0.513783,1000.661179,18.478306,2.102424,2.807739,1000.335845
...,...,...,...,...,...,...,...
995,996,0.435977,1000.528506,18.788124,2.450938,2.984829,1000.433002
996,997,0.334410,1000.433002,18.592034,2.501265,2.239090,1000.184201
997,998,0.536333,1000.184201,18.264480,2.527732,2.991516,1000.660514
998,999,0.802447,1000.660514,18.965601,2.907160,2.853034,1000.267608


In [57]:
#what features constitute a single timeseries sample
group_ids = ['open', 'close', 'high', 'low', 'volume']
target='target'
time_idx = 'date'

max_prediction_length=1 #one day
#list of continous variables that change over time and are not known in the future
time_varying_unknown_reals = ['open', 'close', 'high', 'low', 'volume', 'target']
#normalizer, add later
target_normalizer= TorchNormalizer()

pytorch_random_dataset = TimeSeriesDataSet(random_timeseries,
                                           group_ids=group_ids,
                                           time_idx=time_idx,
                                           target=target,
                                           max_prediction_length=max_prediction_length,
                                           time_varying_unknown_reals=time_varying_unknown_reals,
                                           max_prediction_length = max_prediction_length
                                           )

/home/kagema/anaconda3/envs/csc492_v2/lib/python3.9/site-packages/pytorch_forecasting/data/timeseries.py:1238: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 1000 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__open': 0.20177707821027074, '__group_id__close': 1000.950979287919, '__group_id__high': 18.483054297444216, '__group_id__low': 2.0641541866888065, '__group_id__volume': 2.2564242560663903}, {'__group_id__open': 0.2032789141479468, '__group_id__close': 1000.8677495555643, '__group_id__high': 18.61443423317825, '__group_id__low': 2.9307022308733983, '__group_id__volume': 2.4734635792008732}, {'__group_id__open': 0.20436305172714458, '__group_id__close': 1000.6561191912484, '__group_id__high': 18.932801938271723, '__group_id__low': 2.088536510136771, '__group_id__volume': 2.6125118119636577}, {'__gr

AssertionError: filters should not remove entries all entries - check encoder/decoder lengths and lags

<h3>Transformer Architecture</h3>

Note that the custom encoder contains:
- a self attention layer
- a feed forward layer (normal stuffnot ethat for the attention layer, the multihead attention needs the number of diemnsions to be divisible by the number of multi attention heads.

In [38]:
class StockTransformer(nn.Transformer):

    def __init__(self):
        super().__init__(
            d_model = 6, #pytorch forces the number of dimensions to be divisible by number of md heads
            nhead=3,
            num_encoder_layers=8,
            num_decoder_layers = 8,
            dropout=0.1,
            activation='relu') #not passing everything so we can keep it broken down here

        #how many features are we passing through

        #custom_encoder = some class
        #custom_decoder = some class

        #feed forward already implemented



In [ ]:
#train model
def train(model, data, batch_size=32, optimizer="Adam", learning_rate=0.1, momentum=0.9, num_epochs=10):




In [39]:
model = StockTransformer()

In [40]:
src = torch.rand((10, 32, 6))
tgt = torch.rand((20,32,6))

In [41]:
out = model(src, tgt)

In [ ]:
torch.rand((1,2,3))

In [31]:
out

tensor([[[ 1.8609e+00,  3.5001e-01,  5.2951e-01,  ..., -5.6568e-01,
          -3.1149e-01, -8.8702e-01],
         [ 1.3641e+00,  3.6736e-02,  6.4878e-01,  ...,  4.2819e-01,
          -7.7870e-01,  5.3546e-01],
         [ 1.5085e+00,  6.7698e-01,  5.3151e-01,  ...,  4.6038e-01,
           2.2322e-01, -1.2730e+00],
         ...,
         [ 1.8146e+00,  7.0981e-01,  4.2588e-01,  ...,  9.0969e-01,
           4.8604e-02,  1.7219e-01],
         [ 1.8847e+00,  6.4794e-02,  1.6133e-01,  ...,  1.7033e-01,
          -6.5030e-01,  1.1771e+00],
         [ 1.1817e+00,  6.8779e-01,  5.2459e-01,  ..., -3.5132e-01,
          -3.7258e-01, -5.6009e-02]],

        [[ 1.0215e+00,  7.4425e-01,  1.1591e+00,  ...,  2.7391e-01,
          -2.9277e-01, -1.4802e-01],
         [ 1.9489e+00,  1.6937e-01,  2.9322e-01,  ...,  6.9300e-01,
          -1.0268e+00,  3.5865e-01],
         [ 1.4858e+00,  6.8375e-01,  1.0703e+00,  ...,  7.4776e-01,
           2.1364e-01,  4.6723e-02],
         ...,
         [ 1.9812e+00,  7